In [ ]:
#🔵
main_path = '/Users/stanislavkrupnov/Jup.Notebook'

In [ ]:
#🟠
main_path = '/content/drive/Othercomputers/Mac/Jup.Notebook'
import gdown
from google.colab import drive
drive.mount('/content/drive')

# Описание ноутбука

Я провожу тюнинг гиперпараметров модели **CatBoost Ranker** в две попытки:
1. Используя функцию потерь **Pairlogit**
2. Используя функцию потерь **YetiRank** и метрику **PFound**

Подбор выполняю с помощью Optuna

# Функции

In [ ]:
def metric_eval(predictions: pd.DataFrame, valid: pd.DataFrame,
                id2type: dict) -> tuple:
    """
    Вычисляет метрики для оценки качества модели.

    Parameters:
    - predictions (pd.DataFrame): DataFrame с предсказаниями модели.
    - valid (pd.DataFrame): DataFrame с данными для валидации.
    - id2type (dict): Словарь для преобразования индексов типов в соответствующие строки.

    Returns:
    - tuple: Кортеж, содержащий локальную метрику и реколл для каждого типа.
    """
    # Преобразование индексов типов в соответствующие строки
    valid.type = valid.type.map(lambda idx: id2type[idx])

    # Создание списка с метками для каждой сессии и типа
    ground_truth = valid.groupby(['session', 'type'])['aid'].apply(list)
    ground_truth = ground_truth.reset_index().rename(columns={'aid': 'labels'})

    # Обрезка меток для типа 'clicks' до одной метки
    ground_truth.loc[ground_truth.type == 'clicks',
                     'labels'] = ground_truth.loc[ground_truth.type ==
                                                  'clicks', 'labels'].str[:1]

    # Объединение предсказаний с истинными метками
    submission_with_gt = predictions.merge(
        ground_truth[['session', 'type', 'labels']],
        how='left',
        on=['session', 'type'])

    # Отбрасывание сессий без истинных меток
    submission_with_gt = submission_with_gt[~submission_with_gt.labels_y.isna(
    )]

    # Вычисление количества совпадений между предсказанными и истинными метками
    submission_with_gt['hits'] = submission_with_gt.apply(
        lambda df: len(set(df.labels_x).intersection(set(df.labels_y))),
        axis=1)

    # Вычисление количества истинных меток для каждого типа
    submission_with_gt['gt_count'] = submission_with_gt.labels_y.str.len(
    ).clip(0, 20)

    # Вычисление реколла для каждого типа
    recall_per_type = submission_with_gt.groupby([
        'type'
    ])['hits'].sum() / submission_with_gt.groupby(['type'])['gt_count'].sum()

    # Вычисление локальной метрики на основе реколла для каждого типа
    local_validation_score = (recall_per_type * pd.Series({
        'clicks': 0.10,
        'carts': 0.30,
        'orders': 0.60
    })).sum()

    return local_validation_score, recall_per_type

In [ ]:
def read_parquets(path):
    """
    Чтение данных из файлов Parquet и их объединение в единый DataFrame.

    Parameters:
    - path (str): Путь к файлам Parquet.

    Returns:
    - tr_candidates (pd.DataFrame): Объединенный DataFrame.
    """
    # Список файлов Parquet
    file_list = glob.glob(path)

    # Создание пустого DataFrame для сбора данных
    tr_candidates = pd.DataFrame()

    # Цикл для чтения и объединения файлов
    for file in tqdm(file_list):
        # Чтение файла Parquet во временный DataFrame
        df_ = pd.read_parquet(file)

        # Объединение временного DataFrame с основным датасетом
        tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

    return tr_candidates

# Import

In [ ]:
!pip install optuna

In [ ]:
import os
import sys
from tqdm import tqdm
import glob
import pyarrow.parquet as pq
import pickle
import pandas as pd
import gc
import numpy as np
import itertools
from datetime import datetime as dt
import polars as pl
# import optuna
import shutil
import xgboost as xgb
from sklearn.model_selection import GroupKFold
from imblearn.under_sampling import RandomUnderSampler

# Ver_8 CatB Tune Pairlogit

## Подбор гиперпараметров с Optuna

In [ ]:
ver_tun = 'catb_tun'
ver = 6
ver_folder = f'ver_{ver}'
typee = 'clicks'
# typee = 'carts'
# typee = 'orders'

id2type_name = 'id2type.pkl'

random_state = 42

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
tr_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

100%|██████████| 5/5 [00:22<00:00,  4.49s/it]


**Downsampling**

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# clicks
under_sampler = RandomUnderSampler(
    sampling_strategy=0.43)  # Указываем желаемое соотношение классов

# carts
# under_sampler = RandomUnderSampler(sampling_strategy= 0.0624)  # Указываем желаемое соотношение классов

# orders
# under_sampler = RandomUnderSampler(sampling_strategy= 0.0624)  # Указываем желаемое соотношение классов

In [ ]:
FEATURES = tr_candidates.columns[:-1]
X = tr_candidates.loc[:, FEATURES]
y = tr_candidates.loc[:, 'target']
X_train, y_train = under_sampler.fit_resample(X, y)
tr_candidates = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [ ]:
tr_candidates = tr_candidates.sort_values('session')
tr_candidates = tr_candidates.reset_index(drop = True)

In [ ]:
del X, y, X_train, y_train

In [ ]:
tr_candidates.to_parquet('tr_candidates.parquet')

In [ ]:
tr_candidates = pd.read_parquet('tr_candidates.parquet')

**Optuna**

**Я не могу замерять recall@20 внутри optuna, так как для recall@20 нужно изначально делить сессии на две части, а тут их придется делить внутри фолдов, используя valid части. Посколько это уже не сессии юзеров, а их рекомендации, там только уник. aid, и я не строю внутри фолдов рекомендации опять, а лишь могу  ранжировать, то и конечно пересечений между рекомендациями в одной части и "ист.метками" в другой не будет**. \
Опять же, recall@20 оценивает в целом мою возмонжость рекомендовать, а внутри оптюна моя цель лишь ранжировать хорошо

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 18.3 MB/s eta 0:00:00


In [ ]:
import catboost
from catboost import CatBoostRanker, Pool

In [ ]:
import os
os.environ['CATBOOST_GPU'] = '1'

In [ ]:
folder_path = 'model'
os.makedirs(folder_path, exist_ok=True)

In [ ]:
def objective(trial: optuna.trial.Trial, data: pd.DataFrame,
              random_state: int) -> float:
    """
    Объектная функция для оптимизации гиперпараметров CatBoost.

    Параметры:
    - trial
        Одна итерация оптимизации, содержащая гиперпараметры.
    - data
        Обучающий набор данных.
    - random_state
        Случайное зерно для воспроизводимости.

    Возвращает:
    - Среднее значение метрики pairlogit по всем фолдам для заданных гиперпараметров.
    """

    # Гиперпараметры для оптимизации
    catboost_params = {
        'loss_function':
        'PairLogit',  # Функция потерь PairLogit
        'task_type':
        'GPU',  # Использовать GPU для обучения
        'random_seed':
        trial.suggest_categorical('random_seed',
                                  [random_state]),  # Случайное зерно
        'iterations':
        trial.suggest_categorical("iterations", [1515]),
        'learning_rate':
        trial.suggest_categorical("learning_rate", [0.013688644928808849]),
        'l2_leaf_reg':
        trial.suggest_float('l2_leaf_reg', 1e-5, 1e2,
                            log=True),  # L2 регуляризация
        'border_count':
        trial.suggest_int('border_count', 32,
                          255),  # Количество бинов для числовых признаков
        'random_strength':
        trial.suggest_float('random_strength', 0.0, 1.0),  # Сила случайности
        'bootstrap_type':
        trial.suggest_categorical('bootstrap_type',
                                  ['Bernoulli', 'MVS', 'Poisson']),
        "depth":
        trial.suggest_int("depth", 1, 10),
        "subsample":
        trial.suggest_float("subsample", 0.05, 1.0),
        "min_data_in_leaf":
        trial.suggest_int("min_data_in_leaf", 1, 100)
    }

    # K-fold кросс-валидация
    skf = GroupKFold(n_splits=5)
    FEATURES = data.columns[:-1]

    # Контейнер для хранения значений метрики pairlogit для каждого фолда
    scores = []

    # Итерация по фолдам
    for fold, (train_idx, valid_idx) in enumerate(
            skf.split(data, data['target'], groups=data['session'])):

        X_train = data.loc[train_idx, FEATURES]
        y_train = data.loc[train_idx, 'target']
        X_valid = data.loc[valid_idx, FEATURES]
        y_valid = data.loc[valid_idx, 'target']

        group_id_tr = X_train.iloc[:, 0]
        group_id_v = X_valid.iloc[:, 0]

        X_train = X_train.iloc[:, 2:]
        X_valid = X_valid.iloc[:, 2:]

        catboost_train = Pool(data=X_train,
                              label=y_train,
                              group_id=group_id_tr)
        catboost_valid = Pool(data=X_valid, label=y_valid, group_id=group_id_v)

        # Инициализация CatBoostRanker с гиперпараметрами
        ranker = CatBoostRanker(**catboost_params)

        # Обучение модели и оценка на валидационном наборе
        ranker.fit(catboost_train, eval_set=catboost_valid, verbose_eval=100)

        # Получение метрики pairlogit из лучших результатов валидации
        pairlogit = ranker.best_score_['validation']['PairLogit']

        # Сохранение значения метрики pairlogit для этого фолда
        scores.append(pairlogit)

    # Рассчет среднего значения метрики pairlogit по всем фолдам
    pairlogit_mean = np.mean(scores)

    return pairlogit_mean

In [ ]:
# Создать Optuna study для оптимизации
study = optuna.create_study(direction="minimize",
                            study_name="CatBoost_Ranker_Optimization")
func = lambda trial: objective(trial, tr_candidates, random_state=random_state)

[I 2023-09-29 04:12:26,159] A new study created in memory with name: CatBoost_Ranker_Optimization


In [ ]:
study.optimize(func, n_trials=1, show_progress_bar=True, n_jobs=6)

**Записываю варианты получившихся параметров и выбираю лучшие**

**Clicks** \
0.3941695 \
{'random_seed': 42, 'iterations': 1758, 'learning_rate': 0.053701000860254436} \   
**0.3936051** \
{'random_seed': 42,
 'iterations': 1758,
 'learning_rate': 0.053701000860254436,
 'l2_leaf_reg': 0.8881227642380258,
 'border_count': 92,
 'random_strength': 0.8903326222703105,
 'bootstrap_type': 'Poisson',
 'depth': 6,
 'subsample': 0.9931914618968313,
 'min_data_in_leaf': 7} \.
 **Carts** \
  0.3076646 \   
  {'random_seed': 42, 'iterations': 724, 'learning_rate': 0.04034712260216462} \
  **0.307172** \  
  {'random_seed': 42,
 'iterations': 724,
 'learning_rate': 0.04034712260216462,
 'l2_leaf_reg': 7.449337336120884,
 'border_count': 198,
 'random_strength': 0.3325166004315703,
 'depth': 8,
 'min_data_in_leaf': 12} \  

 **Orders** \
0.1920218 \  
{'random_seed': 42, 'iterations': 1515, 'learning_rate': 0.013688644928808849} \  
0.1920230 \  
{'random_seed': 42,
 'iterations': 1515,
 'learning_rate': 0.013688644928808849,
 'l2_leaf_reg': 15.665624071536678,
 'border_count': 246,
 'random_strength': 0.5474358984091794,
 'depth': 6,
 'min_data_in_leaf': 48} \.  
 **0.1910265** \  
{'random_seed': 42,
 'iterations': 1515,
 'learning_rate': 0.013688644928808849,
 'l2_leaf_reg': 0.03669716203608452,
 'border_count': 235,
 'random_strength': 0.526822845804525,
 'bootstrap_type': 'Poisson',
 'depth': 7,
 'subsample': 0.938154110187783,
 'min_data_in_leaf': 28} \

In [ ]:
study.best_params

{'random_seed': 42,
 'iterations': 1515,
 'learning_rate': 0.013688644928808849,
 'l2_leaf_reg': 0.03669716203608452,
 'border_count': 235,
 'random_strength': 0.526822845804525,
 'bootstrap_type': 'Poisson',
 'depth': 7,
 'subsample': 0.938154110187783,
 'min_data_in_leaf': 28}

**Формирую словари лучших гиперпараметров**

In [ ]:
best_p_cl = {
    'loss_function': 'PairLogit',
    'task_type': 'GPU',
    'random_seed': 42,
    'iterations': 1758,
    'learning_rate': 0.053701000860254436,
    'l2_leaf_reg': 0.8881227642380258,
    'border_count': 92,
    'random_strength': 0.8903326222703105,
    'bootstrap_type': 'Poisson',
    'depth': 6,
    'subsample': 0.9931914618968313,
    'min_data_in_leaf': 7
}

In [ ]:
best_p_carts = {
    'loss_function': 'PairLogit',
    'task_type': 'GPU',
    'random_seed': 42,
    'iterations': 724,
    'learning_rate': 0.04034712260216462,
    'l2_leaf_reg': 7.449337336120884,
    'border_count': 198,
    'random_strength': 0.3325166004315703,
    'depth': 8,
    'min_data_in_leaf': 12
}

In [ ]:
best_p_orders = {
    'loss_function': 'PairLogit',
    'task_type': 'GPU',
    'random_seed': 42,
    'iterations': 1515,
    'learning_rate': 0.013688644928808849,
    'l2_leaf_reg': 0.03669716203608452,
    'border_count': 235,
    'random_strength': 0.526822845804525,
    'bootstrap_type': 'Poisson',
    'depth': 7,
    'subsample': 0.938154110187783,
    'min_data_in_leaf': 28
}

## Обучение на best_params

In [ ]:
downsamp_k = 16
params = [best_p_cl, best_p_carts, best_p_orders]

# Итерация по типам событий ('clicks', 'carts', 'orders') с использованием оптимальных параметров
for typee, best_params in tqdm(zip(['clicks', 'carts', 'orders'], params),
                               desc='type'):

    # Загрузка данных для текущего типа событий
    path = f'{main_path}/tr/ver_6/tr_candidates_{typee}/*'
    tr_candidates = read_parquets(path)
    print('reading_done')

    # Приведение типов данных
    tr_candidates.session = tr_candidates.session.astype(np.int32)
    tr_candidates.aid = tr_candidates.aid.astype(np.int32)

    # Вычисление соотношения классов
    class_ratio = tr_candidates['target'].value_counts()[1] / \
                  tr_candidates['target'].value_counts()[0]

    # Указание желаемого соотношения классов с использованием downsample
    under_sampler = RandomUnderSampler(sampling_strategy=class_ratio *
                                       downsamp_k)

    # Downsampling для лучшего баланса классов
    FEATURES = tr_candidates.columns[:-1]
    X = tr_candidates.loc[:, FEATURES]
    y = tr_candidates.loc[:, 'target']
    X_train, y_train = under_sampler.fit_resample(X, y)
    tr_candidates = pd.merge(X_train,
                             y_train,
                             left_index=True,
                             right_index=True)
    tr_candidates = tr_candidates.sort_values('session')
    tr_candidates = tr_candidates.reset_index(drop=True)
    del X, y
    print('sampling_done')

    # Обучение модели с использованием k-fold кросс-валидации
    skf = GroupKFold(n_splits=5)
    FEATURES = tr_candidates.columns[:-1]
    scores = []

    # Итерация по фолдам
    for fold, (train_idx, valid_idx) in enumerate(
            skf.split(tr_candidates,
                      tr_candidates['target'],
                      groups=tr_candidates['session'])):

        X_train = tr_candidates.loc[train_idx, FEATURES]
        y_train = tr_candidates.loc[train_idx, 'target']
        X_valid = tr_candidates.loc[valid_idx, FEATURES]
        y_valid = tr_candidates.loc[valid_idx, 'target']

        group_id_tr = X_train.iloc[:, 0]
        group_id_v = X_valid.iloc[:, 0]

        X_train = X_train.iloc[:, 2:]
        X_valid = X_valid.iloc[:, 2:]

        catboost_train = Pool(data=X_train,
                              label=y_train,
                              group_id=group_id_tr)
        catboost_valid = Pool(data=X_valid, label=y_valid, group_id=group_id_v)

        # Инициализация CatBoostRanker с оптимальными параметрами
        ranker = CatBoostRanker(**best_params)

        # Обучение модели и оценка на валидационном наборе
        ranker.fit(catboost_train, eval_set=catboost_valid, verbose_eval=100)

        # Рассчет метрики pairlogit
        pairlogit = ranker.best_score_['validation']['PairLogit']
        scores.append(pairlogit)

        # Сохранение модели на диск для каждого фолда
        folder_path = f'model/{ver_tun}'
        os.makedirs(folder_path, exist_ok=True)
        ranker.save_model(f'model/{ver_tun}/catb_tun_{fold}_{typee}')

    # Усреднение метрики pairlogit по всем фолдам
    pairlogit_mean = np.mean(scores)
    print('pairlogit_mean:', pairlogit_mean)
    print('training_done')

type: 0it [00:00, ?it/s]
100%|██████████| 5/5 [00:21<00:00,  4.27s/it]


reading_done
sampling_done
0:	learn: 0.6566164	test: 0.6566426	best: 0.6566426 (0)	total: 45.6ms	remaining: 1m 20s
100:	learn: 0.4008695	test: 0.4023338	best: 0.4023338 (100)	total: 4.31s	remaining: 1m 10s
200:	learn: 0.3970201	test: 0.3989531	best: 0.3989531 (200)	total: 8.66s	remaining: 1m 7s
300:	learn: 0.3952726	test: 0.3976329	best: 0.3976329 (300)	total: 13s	remaining: 1m 2s
400:	learn: 0.3941640	test: 0.3969386	best: 0.3969386 (400)	total: 17.2s	remaining: 58.2s
500:	learn: 0.3932715	test: 0.3964530	best: 0.3964530 (500)	total: 21.4s	remaining: 53.8s
600:	learn: 0.3925190	test: 0.3960877	best: 0.3960877 (600)	total: 25.6s	remaining: 49.3s
700:	learn: 0.3918837	test: 0.3958487	best: 0.3958487 (700)	total: 29.8s	remaining: 44.9s
800:	learn: 0.3912713	test: 0.3956112	best: 0.3956112 (800)	total: 34s	remaining: 40.6s
900:	learn: 0.3906794	test: 0.3954109	best: 0.3954090 (899)	total: 38.1s	remaining: 36.3s
1000:	learn: 0.3901777	test: 0.3952919	best: 0.3952915 (999)	total: 42.3s	rema

type: 1it [08:05, 485.44s/it]

pairlogit_mean: 0.3933920899477553
training_done



100%|██████████| 5/5 [00:33<00:00,  6.73s/it]


...

723:	learn: 0.2887600	test: 0.3069296	best: 0.3069264 (721)	total: 28.2s	remaining: 0us
bestTest = 0.3069264141
bestIteration = 721
Shrink model to first 722 iterations.
pairlogit_mean: 0.30792093498124806
training_done



100%|██████████| 5/5 [00:33<00:00,  6.76s/it]


reading_done
sampling_done
0:	learn: 0.6696325	test: 0.6697260	best: 0.6697260 (0)	total: 35.7ms	remaining: 54.1s
100:	learn: 0.2141224	test: 0.2150934	best: 0.2150934 (100)	total: 3.21s	remaining: 45s
200:	learn: 0.1973599	test: 0.1987737	best: 0.1987737 (200)	total: 6.51s	remaining: 42.6s
300:	learn: 0.1928804	test: 0.1952302	best: 0.1952302 (300)	total: 9.83s	remaining: 39.7s
400:	learn: 0.1901888	test: 0.1934892	best: 0.1934892 (400)	total: 13.2s	remaining: 36.6s
500:	learn: 0.1881997	test: 0.1924409	best: 0.1924409 (500)	total: 16.5s	remaining: 33.3s
600:	learn: 0.1865662	test: 0.1917943	best: 0.1917943 (600)	total: 19.8s	remaining: 30.1s
700:	learn: 0.1851920	test: 0.1913805	best: 0.1913804 (699)	total: 23.1s	remaining: 26.9s
800:	learn: 0.1839145	test: 0.1910064	best: 0.1910062 (799)	total: 26.4s	remaining: 23.6s
900:	learn: 0.1827501	test: 0.1907382	best: 0.1907382 (900)	total: 29.7s	remaining: 20.3s
1000:	learn: 0.1816374	test: 0.1904976	best: 0.1904941 (996)	total: 33s	remain

type: 3it [18:26, 368.79s/it]

1514:	learn: 0.1775457	test: 0.1860902	best: 0.1860873 (1513)	total: 49.9s	remaining: 0us
bestTest = 0.1860873187
bestIteration = 1513
Shrink model to first 1514 iterations.
pairlogit_mean: 0.18968429172051454
training_done


## Pairlogit's каждого типа Cross-val

- pairlogit_mean_clicks: **0.3933920** (Baseline : 0.395109)
- pairlogit_mean_carts: 0.30792093 (Baseline : **0.307252**)
- pairlogit_mean_orders: **0.1896842** (Baseline : 0.191810)

In [ ]:
folder_path = f'{main_path}/models/{ver_tun}/'
os.makedirs(folder_path, exist_ok=True)
file_list = glob.glob(f'/content/model/{ver_tun}/*')
[shutil.copy(file, f'{main_path}/models/{ver_tun}/') for file in file_list]

['/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_0_orders',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_1_clicks',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_4_orders',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_2_carts',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_2_clicks',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_3_clicks',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_4_clicks',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_0_carts',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_2_orders',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_1_carts',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_1_orders',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/cat

## Предсказание + оценка

In [ ]:
lv_predictions_full = pd.DataFrame()

# Итерация по типам событий ('clicks', 'carts', 'orders')
for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    # Проверка наличия кандидатов выбранного типа в рабочей директории
    if not os.path.exists(f'/content/test_candidates_{typee}'):
        # Импорт кандидатов выбранного типа
        shutil.copytree(f'{main_path}/lv/ver_6/test_candidates_{typee}/',
                        f'/content/test_candidates_{typee}/')

    # Загрузка данных для текущего типа событий
    path = f'test_candidates_{typee}/*'
    test_candidates = read_parquets(path)
    test_candidates = test_candidates.sort_values('session')
    test_candidates = test_candidates.reset_index(drop=True)

    # Выделение признаков
    FEATURES = test_candidates.columns[2:]

    # Создание пула данных для CatBoost
    dtest = Pool(data=test_candidates[FEATURES])

    preds = np.zeros(len(test_candidates))

    # Итерация по фолдам для усреднения прогнозов
    for fold in tqdm(range(5)):
        model = CatBoostRanker(random_state=42)
        model.load_model(f'model/{ver_tun}/catb_tun_{fold}_{typee}')
        fold_preds = model.predict(dtest)
        preds += fold_preds / 5

    # Создание DataFrame с прогнозами для текущего типа
    lv_predictions = test_candidates[['session', 'aid']].copy()
    lv_predictions['pred'] = preds
    lv_predictions = lv_predictions.sort_values(
        ['session', 'pred'], ascending=[True, False]).reset_index(drop=True)
    lv_predictions['n'] = lv_predictions.groupby(
        'session').aid.cumcount().astype('int8')
    lv_predictions = lv_predictions.loc[lv_predictions.n < 20]
    lv_predictions = lv_predictions.groupby('session').aid.apply(list)
    lv_predictions = lv_predictions.to_frame().reset_index()
    lv_predictions.rename(columns={'aid': 'labels'}, inplace=True)
    lv_predictions['type'] = typee

    # Объединение временного DataFrame с основным датасетом
    lv_predictions_full = pd.concat([lv_predictions_full, lv_predictions],
                                    ignore_index=True)
    del lv_predictions, preds, test_candidates

    # Сохранение результатов прогнозирования в Parquet-файл
    # lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')
    # shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet')

100%|██████████| 5/5 [00:06<00:00,  1.39s/it]

100%|██████████| 5/5 [00:06<00:00,  1.37s/it]

100%|██████████| 5/5 [00:06<00:00,  1.37s/it]

type: 100%|██████████| 3/3 [06:35<00:00, 131.93s/it]


## Score

In [ ]:
id2type_name = 'id2type.pkl'
# version = 'ver_6.1'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')

with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
    id2type = pickle.load(file)

print('Model score :', metric_eval(lv_predictions_full, valid, id2type))

Model score : (0.5268101513955865, type
carts     0.367607
clicks    0.394414
orders    0.628478
dtype: float64)


- pairlogit_mean_clicks: **0.3933920** (Baseline : 0.395109)
- pairlogit_mean_carts: 0.30792093 (Baseline : **0.307252**)
- pairlogit_mean_orders: **0.1896842** (Baseline : 0.191810)

0.5268101 - если обучать готовую модель на кросс-валидации,  
но бейзлайн оубчался без кросс-валидации.

## Обучение на best_params без кросс-валидации

In [ ]:
best_p_cl = {
    'loss_function': 'PairLogit',
    'task_type': 'GPU',
    'random_seed': 42,
    'iterations': 1758,
    'learning_rate': 0.053701000860254436,
    'l2_leaf_reg': 0.8881227642380258,
    'border_count': 92,
    'random_strength': 0.8903326222703105,
    'bootstrap_type': 'Poisson',
    'depth': 6,
    'subsample': 0.9931914618968313,
    'min_data_in_leaf': 7
}

In [ ]:
best_p_carts = {
    'loss_function': 'PairLogit',
    'task_type': 'GPU',
    'random_seed': 42,
    'iterations': 724,
    'learning_rate': 0.04034712260216462,
    'l2_leaf_reg': 7.449337336120884,
    'border_count': 198,
    'random_strength': 0.3325166004315703,
    'depth': 8,
    'min_data_in_leaf': 12
}

In [ ]:
best_p_orders = {
    'loss_function': 'PairLogit',  # Функция потерь PairLogit
    'task_type': 'GPU',
    'random_seed': 42,
    'iterations': 1515,
    'learning_rate': 0.013688644928808849,
    'l2_leaf_reg': 0.03669716203608452,
    'border_count': 235,
    'random_strength': 0.526822845804525,
    'bootstrap_type': 'Poisson',
    'depth': 7,
    'subsample': 0.938154110187783,
    'min_data_in_leaf': 28
}

In [ ]:
downsamp_k = 16
params = [best_p_cl, best_p_carts, best_p_orders]
for typee, best_params in tqdm(zip(['clicks', 'carts', 'orders'], params),
                               desc='type'):

    path = f'{main_path}/tr/ver_6/tr_candidates_{typee}/*'
    tr_candidates = read_parquets(path)
    print('reading_done')

    tr_candidates.session = tr_candidates.session.astype(np.int32)
    tr_candidates.aid = tr_candidates.aid.astype(np.int32)

    class_ratio = tr_candidates['target'].value_counts()[1] / \
                  tr_candidates['target'].value_counts()[0]

    # Указываем желаемое соотношение классов
    under_sampler = RandomUnderSampler(sampling_strategy=class_ratio *
                                       downsamp_k)

    # downsamling для лучшего баланса классов
    FEATURES = tr_candidates.columns[:-1]
    X = tr_candidates.loc[:, FEATURES]
    y = tr_candidates.loc[:, 'target']
    X_train, y_train = under_sampler.fit_resample(X, y)
    tr_candidates = pd.merge(X_train,
                             y_train,
                             left_index=True,
                             right_index=True)
    tr_candidates = tr_candidates.sort_values('session')
    tr_candidates = tr_candidates.reset_index(drop=True)
    del X, y
    print('sampling_done')

    # Обучение модели
    X_train = tr_candidates[FEATURES]
    y_train = tr_candidates['target']

    group_id_tr = X_train.iloc[:, 0]

    X_train = X_train.iloc[:, 2:]

    catboost_train = Pool(data=X_train, label=y_train, group_id=group_id_tr)

    ranker = CatBoostRanker(**best_params)
    ranker.fit(catboost_train, early_stopping_rounds=50, verbose=False)

    # Рассчитать метрику pairlogit
    pairlogit = ranker.best_score_['learn']['PairLogit']

    folder_path = f'model/{ver_tun}_no_kfold'
    os.makedirs(folder_path, exist_ok=True)
    ranker.save_model(f'model/{ver_tun}_no_kfold/catb_tun_{fold}_{typee}')

    print('pairlogit:', pairlogit)
    print('training_done')

type: 0it [00:00, ?it/s]
100%|██████████| 5/5 [00:23<00:00,  4.65s/it]


reading_done
sampling_done


type: 1it [02:51, 171.05s/it]

pairlogit: 0.3883519543825259
training_done



100%|██████████| 5/5 [00:31<00:00,  6.22s/it]


...

pairlogit: 0.29182062795452757
training_done



100%|██████████| 5/5 [00:30<00:00,  6.16s/it]


reading_done
sampling_done


type: 3it [07:23, 147.89s/it]

pairlogit: 0.18069645427941486
training_done


In [ ]:
ranker.best_score_

{'learn': {'PairLogit': 0.3890073754283778}}

## Предсказание + оценка (повторное)

In [ ]:
lv_predictions_full = pd.DataFrame()

# Итерация по типам событий ('clicks', 'carts', 'orders')
for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    # Проверка наличия кандидатов выбранного типа в рабочей директории
    if not os.path.exists(f'/content/test_candidates_{typee}'):
        # Импорт кандидатов выбранного типа
        shutil.copytree(f'{main_path}/lv/ver_6/test_candidates_{typee}/',
                        f'/content/test_candidates_{typee}/')

    # Загрузка данных для текущего типа событий
    path = f'test_candidates_{typee}/*'
    test_candidates = read_parquets(path)
    test_candidates = test_candidates.sort_values('session')
    test_candidates = test_candidates.reset_index(drop=True)

    # Выделение признаков
    FEATURES = test_candidates.columns[2:]

    # Создание пула данных для CatBoost
    dtest = Pool(data=test_candidates[FEATURES])

    # Загрузка обученной модели для текущего типа событий (в данном случае, для 4-го фолда)
    model = CatBoostRanker(random_state=42)
    model.load_model(f'model/{ver_tun}/catb_tun_4_{typee}')

    # Получение прогнозов
    preds = model.predict(dtest)

    # Создание DataFrame с прогнозами для текущего типа
    lv_predictions = test_candidates[['session', 'aid']].copy()
    lv_predictions['pred'] = preds
    lv_predictions = lv_predictions.sort_values(
        ['session', 'pred'], ascending=[True, False]).reset_index(drop=True)
    lv_predictions['n'] = lv_predictions.groupby(
        'session').aid.cumcount().astype('int8')
    lv_predictions = lv_predictions.loc[lv_predictions.n < 20]
    lv_predictions = lv_predictions.groupby('session').aid.apply(list)
    lv_predictions = lv_predictions.to_frame().reset_index()
    lv_predictions.rename(columns={'aid': 'labels'}, inplace=True)
    lv_predictions['type'] = typee

    # Объединение временного DataFrame с основным датасетом
    lv_predictions_full = pd.concat([lv_predictions_full, lv_predictions],
                                    ignore_index=True)
    del lv_predictions, preds, test_candidates

    # Сохранение результатов прогнозирования в Parquet-файл
    # lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')
    # shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet')

type: 100%|██████████| 3/3 [03:45<00:00, 75.19s/it]


## Score

In [ ]:
id2type_name = 'id2type.pkl'
# version = 'ver_6.1'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')

with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
    id2type = pickle.load(file)

print('Model score :', metric_eval(lv_predictions_full, valid, id2type))

Model score : (0.52662368294355, type
carts     0.367851
clicks    0.393613
orders    0.628179
dtype: float64)


0.5268101 - если обучать готовую модель на кросс-валидации,  
но бейзлайн оубчался без кросс-валидации.


# Ver_8 CatB Tune PFound

## Подбор гиперпараметров с Optuna

In [ ]:
ver_tun = 'catb_tun'
ver = 6
ver_folder = f'ver_{ver}'
# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

id2type_name = 'id2type.pkl'

random_state = 42

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
tr_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

100%|██████████| 5/5 [00:36<00:00,  7.25s/it]


**Downsampling**

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# clicks
# under_sampler = RandomUnderSampler(sampling_strategy=0.43)  # Указываем желаемое соотношение классов

# carts
under_sampler = RandomUnderSampler(sampling_strategy= 0.0624)  # Указываем желаемое соотношение классов

# orders
# under_sampler = RandomUnderSampler(sampling_strategy= 0.0624)  # Указываем желаемое соотношение классов

In [ ]:
FEATURES = tr_candidates.columns[:-1]
X = tr_candidates.loc[:, FEATURES]
y = tr_candidates.loc[:, 'target']
X_train, y_train = under_sampler.fit_resample(X, y)
tr_candidates = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [ ]:
tr_candidates = tr_candidates.sort_values('session')
tr_candidates = tr_candidates.reset_index(drop = True)

In [ ]:
del X, y, X_train, y_train

**Optuna**

In [ ]:
!pip install catboost

  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/a2/aa/1f6a0ae3224f11bafb195035f56bb0fc99ed948cca3c052138665d73801c/catboost-1.2.2-cp311-cp311-macosx_11_0_universal2.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 423.8 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━ 18.0/25.7 MB 10.7 MB/s eta 0:00:01

limit_output extension: Maximum message size of 10000 exceeded with 10129 characters

In [ ]:
import catboost
from catboost import CatBoostRanker, Pool

In [ ]:
import os
os.environ['CATBOOST_GPU'] = '1'

In [ ]:
folder_path = 'model'
os.makedirs(folder_path, exist_ok=True)

**Очень странно - чтобы вытащить метрику PFound для return в objective - нужно указать в catboost_params custom_metric = 'Pairlogit', но не PFound. Но потом она оказывается таки в слвоаре best_score_, и я могу ее брать и выдавать для оптимизации. Написал запрос в поддержку CatBoost**

In [ ]:
def objective(trial, data, random_state):

    catboost_params = {
        'loss_function':
        'YetiRank:hints=skip_train~false',
        # 'loss_function': 'PairLogit',# Функция потерь PairLogit
        # 'custom_metric': 'PFound:hints=skip_train~false',
        'custom_metric':
        'PairLogit',
        'task_type':
        'GPU',  # Использовать GPU
        'metric_period':
        250,
        'random_seed':
        trial.suggest_categorical('random_seed',
                                  [random_state]),  # Случайное зерно
        'iterations':
        trial.suggest_categorical("iterations", [828]),
        'learning_rate':
        trial.suggest_categorical("learning_rate", [0.12676718423508196]),
        # 'iterations':
#         trial.suggest_int('iterations', 500, 2000),  # Количество итераций
        # 'learning_rate':
#         trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        'l2_leaf_reg':
        trial.suggest_float('l2_leaf_reg', 1e-5, 1e2,
                            log=True),  # L2 регуляризация
        'border_count':
        trial.suggest_int('border_count', 32,
                          255),  # Количество бинов для числовых признаков
        'random_strength':
        trial.suggest_float('random_strength', 0.0, 1.0),  # Сила случайности
        # 'bootstrap_type':
#         trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'MVS', 'Poisson']),
        "depth":
        trial.suggest_int("depth", 1, 10),
        # "subsample":
#         trial.suggest_float("subsample", 0.05, 1.0),
        "min_data_in_leaf":
        trial.suggest_int("min_data_in_leaf", 1, 100)
    }

    # data = tr_candidates
    skf = GroupKFold(n_splits=5)
    FEATURES = data.columns[:-1]

    scores = []

    for fold, (train_idx, valid_idx) in enumerate(
            skf.split(data, data['target'], groups=data['session'])):

        X_train = data.loc[train_idx, FEATURES]
        y_train = data.loc[train_idx, 'target']
        X_valid = data.loc[valid_idx, FEATURES]
        y_valid = data.loc[valid_idx, 'target']

        group_id_tr = X_train.iloc[:, 0]
        group_id_v = X_valid.iloc[:, 0]

        X_train = X_train.iloc[:, 2:]
        X_valid = X_valid.iloc[:, 2:]

        catboost_train = Pool(data=X_train,
                              label=y_train,
                              group_id=group_id_tr)
        catboost_valid = Pool(data=X_valid, label=y_valid, group_id=group_id_v)

        ranker = CatBoostRanker(**catboost_params)

        ranker.fit(catboost_train, eval_set=catboost_valid, verbose_eval=250)

        # Рассчитать метрику PFound
        print(ranker.best_score_)
        PFound = ranker.best_score_['validation']['PFound']
        # PFound = ranker.best_score_['validation']['PairLogit']
        scores.append(PFound)

    # Усреднить PFound по всем фолдам
    PFound_mean = np.mean(scores)

    return PFound_mean

In [ ]:
# Создать Optuna study для оптимизации
study = optuna.create_study(direction="maximize",
                            study_name="CatBoost_Ranker_Optimization")
func = lambda trial: objective(trial, tr_candidates, random_state=random_state)

[I 2023-10-03 03:08:48,011] A new study created in memory with name: CatBoost_Ranker_Optimization


In [ ]:
study.optimize(func, n_trials=1, show_progress_bar=True, n_jobs=6)

  0%|          | 0/1 [00:00<?, ?it/s]

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.0921486	best: 0.0921486 (0)	total: 91.7ms	remaining: 1m 15s
250:	test: 0.0938066	best: 0.0938066 (250)	total: 9.21s	remaining: 21.2s
500:	test: 0.0938388	best: 0.0938388 (500)	total: 18.4s	remaining: 12s
750:	test: 0.0938526	best: 0.0938526 (750)	total: 27.7s	remaining: 2.83s
827:	test: 0.0938592	best: 0.0938592 (827)	total: 30.5s	remaining: 0us
bestTest = 0.09385917151
bestIteration = 827
{'learn': {'PairLogit': 0.19596942640881637}, 'validation': {'PFound': 0.093859171511442, 'PairLogit': 0.1967569390450675}}


Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.0926297	best: 0.0926297 (0)	total: 92.6ms	remaining: 1m 16s
250:	test: 0.0941957	best: 0.0941957 (250)	total: 9.21s	remaining: 21.2s
500:	test: 0.0942279	best: 0.0942279 (500)	total: 18.4s	remaining: 12s
750:	test: 0.0942356	best: 0.0942356 (750)	total: 27.6s	remaining: 2.83s
827:	test: 0.0942496	best: 0.0942496 (827)	total: 30.4s	remaining: 0us
bestTest = 0.09424962198
bestIteration = 827
{'learn': {'PairLogit': 0.19594987703196293}, 'validation': {'PFound': 0.09424962198143552, 'PairLogit': 0.19638691812244805}}


...

0:	test: 0.0921410	best: 0.0921410 (0)	total: 87.6ms	remaining: 1m 12s
250:	test: 0.0937905	best: 0.0937905 (250)	total: 9.17s	remaining: 21.1s
500:	test: 0.0938374	best: 0.0938374 (500)	total: 18.3s	remaining: 12s
750:	test: 0.0938671	best: 0.0938671 (750)	total: 27.5s	remaining: 2.82s
827:	test: 0.0938646	best: 0.0938671 (750)	total: 30.4s	remaining: 0us
bestTest = 0.09386705738
bestIteration = 750
Shrink model to first 751 iterations.
{'learn': {'PairLogit': 0.19430743095612654}, 'validation': {'PFound': 0.09386705738080897, 'PairLogit': 0.2034723882991134}}


Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.0917060	best: 0.0917060 (0)	total: 93.5ms	remaining: 1m 17s
250:	test: 0.0933459	best: 0.0933459 (250)	total: 9.2s	remaining: 21.1s
500:	test: 0.0933894	best: 0.0933894 (500)	total: 18.4s	remaining: 12s
750:	test: 0.0934004	best: 0.0934004 (750)	total: 27.6s	remaining: 2.83s
827:	test: 0.0934061	best: 0.0934061 (827)	total: 30.4s	remaining: 0us
bestTest = 0.09340612441
bestIteration = 827
{'learn': {'PairLogit': 0.19562670159185774}, 'validation': {'PFound': 0.0934061244060777, 'PairLogit': 0.19730977283221143}}


Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.0922445	best: 0.0922445 (0)	total: 87.5ms	remaining: 1m 12s
250:	test: 0.0938771	best: 0.0938771 (250)	total: 9.16s	remaining: 21.1s
500:	test: 0.0939175	best: 0.0939175 (500)	total: 18.3s	remaining: 12s
750:	test: 0.0939342	best: 0.0939342 (750)	total: 27.5s	remaining: 2.82s
827:	test: 0.0939437	best: 0.0939437 (827)	total: 30.4s	remaining: 0us
bestTest = 0.09394374069
bestIteration = 827
{'learn': {'PairLogit': 0.1958391153037997}, 'validation': {'PFound': 0.0939437406903854, 'PairLogit': 0.19701879633920297}}
[I 2023-10-02 08:35:23,152] Trial 0 finished with value: 0.09386514319402992 and parameters: {'random_seed': 42, 'iterations': 828, 'learning_rate': 0.12676718423508196, 'l2_leaf_reg': 0.002270370592919817, 'border_count': 37, 'random_strength': 0.9389551921967575, 'depth': 6, 'min_data_in_leaf': 73}. Best is trial 0 with value: 0.09386514319402992.


**Записываю варианты получившихся параметров и выбираю лучшие**

**Clicks** \
0.6159558 \
{'random_seed': 42, 'iterations': 800, 'learning_rate': 0.09237673064613201} \   
**0.616068** \
{'random_seed': 42,
 'iterations': 800,
 'learning_rate': 0.09237673064613201,
 'l2_leaf_reg': 0.0019442470574252018,
 'border_count': 97,
 'random_strength': 0.5971699972467241,
 'depth': 7,
 'min_data_in_leaf': 100} \
 **Carts** \
  0.150190 \   
{'random_seed': 42, 'iterations': 551, 'learning_rate': 0.010372549720642329} \
  **0.1502257** \  
  'random_seed': 42,
 'iterations': 551,
 'learning_rate': 0.010372549720642329,
 'l2_leaf_reg': 0.019335935779919735,
 'border_count': 175,
 'random_strength': 0.9145153914960099,
 'bootstrap_type': 'MVS',
 'depth': 10,
 'subsample': 0.2884983050620958,
 'min_data_in_leaf': 66 \

 **Orders** \
0.093870 \  
{'random_seed': 42, 'iterations': 828, 'learning_rate': 0.12676718423508196}\.  
**0.093865**  \   
{'random_seed': 42,
 'iterations': 828,
 'learning_rate': 0.12676718423508196,
 'l2_leaf_reg': 0.002270370592919817,
 'border_count': 37,
 'random_strength': 0.9389551921967575,
 'depth': 6,
 'min_data_in_leaf': 73} \



In [ ]:
study.best_params

{'random_seed': 42,
 'iterations': 828,
 'learning_rate': 0.12676718423508196,
 'l2_leaf_reg': 0.002270370592919817,
 'border_count': 37,
 'random_strength': 0.9389551921967575,
 'depth': 6,
 'min_data_in_leaf': 73}

**Формирую словари лучших гиперпараметров**

In [ ]:
best_p_cl = {
    'loss_function': 'YetiRank:hints=skip_train~false',
    'custom_metric': 'PairLogit',
    'task_type': 'GPU',
    'random_seed': 42,
    'iterations': 800,
    'learning_rate': 0.09237673064613201,
    'l2_leaf_reg': 0.0019442470574252018,
    'border_count': 97,
    'random_strength': 0.5971699972467241,
    'depth': 7,
    'min_data_in_leaf': 100
}

In [ ]:
best_p_carts = {
    'loss_function': 'YetiRank:hints=skip_train~false',
    'custom_metric': 'PairLogit',
    'task_type': 'GPU',
    'random_seed': 42,
    'iterations': 551,
    'learning_rate': 0.010372549720642329,
    'l2_leaf_reg': 0.019335935779919735,
    'border_count': 175,
    'random_strength': 0.9145153914960099,
    'bootstrap_type': 'MVS',
    'depth': 10,
    'subsample': 0.2884983050620958,
    'min_data_in_leaf': 66
}

In [ ]:
best_p_orders = {
    'loss_function': 'YetiRank:hints=skip_train~false',
    'custom_metric': 'PairLogit',
    'task_type': 'GPU',
    'random_seed': 42,
    'random_seed': 42,
    'iterations': 828,
    'learning_rate': 0.12676718423508196,
    'l2_leaf_reg': 0.002270370592919817,
    'border_count': 37,
    'random_strength': 0.9389551921967575,
    'depth': 6,
    'min_data_in_leaf': 73
}

## Обучение на best_params

In [ ]:
downsamp_k = 16
params = [best_p_cl, best_p_carts, best_p_orders]

# Итерация по типам событий ('clicks', 'carts', 'orders') и их лучшим параметрам
for typee, best_params in tqdm(zip(['clicks', 'carts', 'orders'], params),
                               desc='type'):

    # Загрузка данных для текущего типа событий
    path = f'{main_path}/tr/ver_6/tr_candidates_{typee}/*'
    tr_candidates = read_parquets(path)
    print('reading_done')

    # Приведение типов данных для сессии и aid
    tr_candidates.session = tr_candidates.session.astype(np.int32)
    tr_candidates.aid = tr_candidates.aid.astype(np.int32)

    # Вычисление соотношения классов
    class_ratio = tr_candidates['target'].value_counts()[1] / \
                  tr_candidates['target'].value_counts()[0]

    # Указываем желаемое соотношение классов и проводим downsampling
    under_sampler = RandomUnderSampler(sampling_strategy=class_ratio *
                                       downsamp_k)
    FEATURES = tr_candidates.columns[:-1]
    X = tr_candidates.loc[:, FEATURES]
    y = tr_candidates.loc[:, 'target']
    X_train, y_train = under_sampler.fit_resample(X, y)
    tr_candidates = pd.merge(X_train,
                             y_train,
                             left_index=True,
                             right_index=True)
    tr_candidates = tr_candidates.sort_values('session')
    tr_candidates = tr_candidates.reset_index(drop=True)
    del X, y
    print('sampling_done')

    # Обучение модели с использованием GroupKFold
    skf = GroupKFold(n_splits=5)
    FEATURES = tr_candidates.columns[:-1]

    scores = []

    for fold, (train_idx, valid_idx) in enumerate(
            skf.split(tr_candidates,
                      tr_candidates['target'],
                      groups=tr_candidates['session'])):

        X_train = tr_candidates.loc[train_idx, FEATURES]
        y_train = tr_candidates.loc[train_idx, 'target']
        X_valid = tr_candidates.loc[valid_idx, FEATURES]
        y_valid = tr_candidates.loc[valid_idx, 'target']

        group_id_tr = X_train.iloc[:, 0]
        group_id_v = X_valid.iloc[:, 0]

        X_train = X_train.iloc[:, 2:]
        X_valid = X_valid.iloc[:, 2:]

        catboost_train = Pool(data=X_train,
                              label=y_train,
                              group_id=group_id_tr)
        catboost_valid = Pool(data=X_valid, label=y_valid, group_id=group_id_v)

        # Создание и обучение CatBoostRanker с лучшими параметрами
        ranker = CatBoostRanker(**best_params)
        ranker.fit(catboost_train, eval_set=catboost_valid, verbose_eval=100)

        # Рассчитать метрику PFound
        PFound = ranker.best_score_['validation']['PFound']
        scores.append(PFound)

        # Сохранение модели на диск
        folder_path = f'model/{ver_tun}'
        os.makedirs(folder_path, exist_ok=True)
        ranker.save_model(f'model/{ver_tun}/catb_tun_{fold}_{typee}')

    # Усреднить PFound по всем фолдам
    PFound_mean = np.mean(scores)
    print('PFound_mean:', PFound_mean)
    print('training_done')

type: 0it [00:00, ?it/s]
100%|██████████| 5/5 [00:33<00:00,  6.79s/it]


reading_done
sampling_done


Default metric period is 5 because PFound is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.5977711	best: 0.5977711 (0)	total: 173ms	remaining: 2m 17s
100:	test: 0.6157592	best: 0.6157592 (100)	total: 6.61s	remaining: 45.8s
200:	test: 0.6164299	best: 0.6164299 (200)	total: 13s	remaining: 38.7s
300:	test: 0.6167142	best: 0.6167147 (295)	total: 19.4s	remaining: 32.1s
400:	test: 0.6169380	best: 0.6169380 (400)	total: 25.8s	remaining: 25.7s
500:	test: 0.6170515	best: 0.6170515 (500)	total: 32.3s	remaining: 19.3s
600:	test: 0.6171167	best: 0.6171167 (600)	total: 38.9s	remaining: 12.9s
700:	test: 0.6171755	best: 0.6171823 (675)	total: 45.5s	remaining: 6.42s
799:	test: 0.6172082	best: 0.6172082 (799)	total: 52s	remaining: 0us
bestTest = 0.6172081932
bestIteration = 799
{'learn': {'PairLogit': 0.4010464509089207}, 'validation': {'PFound': 0.6172081932040321, 'PairLogit': 0.40130148153235745}}


Default metric period is 5 because PFound is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

0:	test: 0.0885629	best: 0.0885629 (0)	total: 111ms	remaining: 1m 32s
100:	test: 0.0897535	best: 0.0897535 (100)	total: 4.88s	remaining: 35.1s
200:	test: 0.0898064	best: 0.0898083 (195)	total: 9.69s	remaining: 30.2s
300:	test: 0.0898464	best: 0.0898484 (295)	total: 14.5s	remaining: 25.4s
400:	test: 0.0898633	best: 0.0898652 (390)	total: 19.3s	remaining: 20.6s
500:	test: 0.0898809	best: 0.0898809 (500)	total: 24.1s	remaining: 15.7s
600:	test: 0.0898950	best: 0.0898968 (590)	total: 28.9s	remaining: 10.9s
700:	test: 0.0899014	best: 0.0899021 (695)	total: 33.8s	remaining: 6.13s
800:	test: 0.0899032	best: 0.0899064 (780)	total: 38.7s	remaining: 1.3s
827:	test: 0.0899071	best: 0.0899078 (820)	total: 40s	remaining: 0us
bestTest = 0.08990775654
bestIteration = 820
Shrink model to first 821 iterations.
{'learn': {'PairLogit': 0.1956116568691615}, 'validation': {'PFound': 0.08990775653754342, 'PairLogit': 0.19384133155460914}}


Default metric period is 5 because PFound is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.0870828	best: 0.0870828 (0)	total: 118ms	remaining: 1m 37s
100:	test: 0.0888922	best: 0.0888922 (100)	total: 4.87s	remaining: 35.1s
200:	test: 0.0889489	best: 0.0889489 (200)	total: 9.66s	remaining: 30.1s
300:	test: 0.0889819	best: 0.0889819 (300)	total: 14.5s	remaining: 25.3s
400:	test: 0.0890005	best: 0.0890005 (400)	total: 19.3s	remaining: 20.5s
500:	test: 0.0890093	best: 0.0890103 (490)	total: 24.1s	remaining: 15.7s
600:	test: 0.0890150	best: 0.0890181 (580)	total: 28.9s	remaining: 10.9s
700:	test: 0.0890157	best: 0.0890199 (685)	total: 33.7s	remaining: 6.11s
800:	test: 0.0890323	best: 0.0890323 (800)	total: 38.5s	remaining: 1.3s


type: 3it [16:15, 325.33s/it]

827:	test: 0.0890326	best: 0.0890341 (825)	total: 39.8s	remaining: 0us
bestTest = 0.08903407522
bestIteration = 825
Shrink model to first 826 iterations.
{'learn': {'PairLogit': 0.19447849352610785}, 'validation': {'PFound': 0.08903407522394335, 'PairLogit': 0.19846317343904588}}
PFound_mean: 0.08957375668481923
training_done


## Pairlogit's каждого типа Cross-val

- pairlogit_mean_clicks: **0.3933920** (Baseline : 0.395109)
- pairlogit_mean_carts: 0.30792093 (Baseline : **0.307252**)
- pairlogit_mean_orders: **0.1896842** (Baseline : 0.191810)

In [ ]:
folder_path = f'{main_path}/models/{ver_tun}/'
os.makedirs(folder_path, exist_ok=True)
file_list = glob.glob(f'/content/model/{ver_tun}/*')
[shutil.copy(file, f'{main_path}/models/{ver_tun}/') for file in file_list]

['/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_0_orders',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_1_clicks',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_4_orders',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_2_carts',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_2_clicks',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_3_clicks',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_4_clicks',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_0_carts',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_2_orders',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_1_carts',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/catb_tun_1_orders',
 '/content/drive/Othercomputers/Mac/Jup.Notebook/models/catb_tun/cat

## Предсказание + оценка

**На тестовых данных**

In [ ]:
lv_predictions_full = pd.DataFrame()

# Итерация по типам событий ('clicks', 'carts', 'orders')
for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    # Проверка наличия кандидатов выбранного типа в нужном месте
    if os.path.exists(f'/content/test_candidates_{typee}') == False:
        # Импорт кандидатов выбранного типа
        shutil.copytree(f'{main_path}/lv/ver_6/test_candidates_{typee}/',
                        f'/content/test_candidates_{typee}/')

    # Загрузка данных для текущего типа событий
    path = f'test_candidates_{typee}/*'
    test_candidates = read_parquets(path)
    test_candidates = test_candidates.sort_values('session')
    test_candidates = test_candidates.reset_index(drop=True)

    FEATURES = test_candidates.columns[2:]
    dtest = Pool(data=test_candidates[FEATURES])
    preds = np.zeros(len(test_candidates))

    # Итерация по фолдам (в данном случае 5)
    for fold in tqdm(range(5)):
        model = CatBoostRanker(random_state=42)
        model.load_model(f'models/{ver_tun}/catb_tun_{fold}_{typee}')
        fold_preds = model.predict(dtest)
        preds += fold_preds / 5

    # Создание DataFrame для предсказаний текущего типа событий
    lv_predictions = test_candidates[['session', 'aid']].copy()
    lv_predictions['pred'] = preds
    lv_predictions = lv_predictions.sort_values(
        ['session', 'pred'], ascending=[True, False]).reset_index(drop=True)
    lv_predictions['n'] = lv_predictions.groupby(
        'session').aid.cumcount().astype('int8')
    lv_predictions = lv_predictions.loc[lv_predictions.n < 20]
    lv_predictions = lv_predictions.groupby('session').aid.apply(list)
    lv_predictions = lv_predictions.to_frame().reset_index()
    lv_predictions.rename(columns={'aid': 'labels'}, inplace=True)
    lv_predictions['type'] = typee

    # Объединение временного DataFrame с основным датасетом
    lv_predictions_full = pd.concat([lv_predictions_full, lv_predictions],
                                    ignore_index=True)
    del lv_predictions, preds, test_candidates

100%|██████████| 5/5 [00:07<00:00,  1.42s/it]

100%|██████████| 5/5 [00:06<00:00,  1.38s/it]

100%|██████████| 5/5 [00:07<00:00,  1.42s/it]

type: 100%|██████████| 3/3 [05:54<00:00, 118.02s/it]


**На тренировочных данных**

In [ ]:
train_predictions_full = pd.DataFrame()

for typee in tqdm(['clicks', 'carts', 'orders'], desc='type'):

    path = f"{main_path}/tr/ver_6/tr_candidates_{typee}/*"
    train_candidates = read_parquets(path)
    train_candidates = train_candidates.sort_values('session')
    train_candidates = train_candidates.reset_index(drop=True)

    FEATURES = train_candidates.columns[2:-1]
    dtest = Pool(data=train_candidates[FEATURES])
    preds = np.zeros(len(train_candidates))

    for fold in tqdm(range(5)):
        model = CatBoostRanker(random_state=42)
        model.load_model(f'models/{ver_tun}/catb_tun_{fold}_{typee}')
        fold_preds = model.predict(dtest)
        preds += fold_preds / 5

    lv_predictions = train_candidates[['session', 'aid']].copy()
    lv_predictions['pred'] = preds
    lv_predictions = lv_predictions.sort_values(
        ['session', 'pred'], ascending=[True, False]).reset_index(drop=True)
    lv_predictions['n'] = lv_predictions.groupby(
        'session').aid.cumcount().astype('int8')
    lv_predictions = lv_predictions.loc[lv_predictions.n < 20]
    lv_predictions = lv_predictions.groupby('session').aid.apply(list)
    lv_predictions = lv_predictions.to_frame().reset_index()
    lv_predictions.rename(columns={'aid': 'labels'}, inplace=True)
    lv_predictions['type'] = typee

    # Объединение временного DataFrame с основным датасетом
    train_predictions_full = pd.concat(
        [train_predictions_full, lv_predictions], ignore_index=True)
    del lv_predictions, preds, train_candidates

100%|█████████████████████████████████████████████| 5/5 [00:20<00:00,  4.17s/it]

100%|█████████████████████████████████████████████| 5/5 [00:16<00:00,  3.33s/it]

100%|█████████████████████████████████████████████| 5/5 [00:16<00:00,  3.32s/it]

type: 100%|██████████████████████████████████████| 3/3 [12:45<00:00, 255.30s/it]


## Score

**Test**

In [ ]:
id2type_name = 'id2type.pkl'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')

with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
    id2type = pickle.load(file)

print('Model score :', metric_eval(lv_predictions_full, valid, id2type))

Model score : (0.5272565979917239, type
carts     0.367094
clicks    0.398173
orders    0.628852
dtype: float64)


**Train**

In [ ]:
# В роли test_labels испольую таргеты, созданные для обучения на этом тренировочном
# датасете (на котором я выше предсказался)
test_labels = pd.read_parquet(f'{main_path}/s/targets.parquet')

with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
    id2type = pickle.load(file)

print('Model score :', metric_eval(train_predictions_full, test_labels,
                                   id2type))

Model score : (0.5272965939417337, type
carts     0.378590
clicks    0.394375
orders    0.624489
dtype: float64)


In [ ]:
print(
    abs(0.5272965939417337 - 0.5272565979917239) / 0.5272565979917239 * 100,
    '%')

0.007585670840753659 %


**Модель не переобучилась, разница в метрике между трейн и тест = 0.007%**

# Вывод

CatBoost Ranker Baseline метрика = 0.527116  
**YetiRank(PFound)** обошел по метрике Pairlogit на тюнинге  
Tuning (PFound в качестве прокси метрики) метрика = **0.527256**  
Отлично, двигаюсь далее к стекингу